<a href="https://colab.research.google.com/github/Saultr21/IA-Y-BIGDATA/blob/main/PRO/CartPole/Untitled57.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 INSTALAR DEPENDENCIAS EN GOOGLE COLAB
!sudo apt-get update --fix-missing
!sudo apt-get install -y xvfb ffmpeg
!pip install -U gym
!pip install pygame
!pip install keras
!pip install tensorflow
!pip install pyvirtualdisplay

# 🎥 HABILITAR EL RENDERIZADO EN COLAB
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

print("¡Virtual Display iniciado correctamente!")

# 📌 IMPORTAR LIBRERÍAS NECESARIAS
import numpy as np
import pandas as pd
import gym
import random
import cv2
import base64
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from IPython.display import HTML

# 📌 REGISTRAR MÉTRICAS PARA TENSORBOARD
tensorboard_callback = TensorBoard(log_dir="./logs")

# 🔥 DEFINICIÓN DEL AGENTE DQL
class DQLAgent():
    def __init__(self, env):
        self.state_size = env.observation_space.shape[0]
        self.action_size = env.action_space.n
        self.gamma = 0.95
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.memory = deque(maxlen=1000)
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(48, input_dim=self.state_size, activation='tanh'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if random.uniform(0,1) <= self.epsilon:
            return env.action_space.sample()
        else:
            act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward if done else reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0])
            train_target = self.model.predict(state, verbose=0)
            train_target[0][action] = target
            self.model.fit(state, train_target, verbose=0, callbacks=[tensorboard_callback])

    def adaptiveEGreedy(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# 📌 INICIALIZAR EL ENTORNO
env = gym.make('CartPole-v1', render_mode="rgb_array")

# 📌 ENTRENAMIENTO DEL AGENTE
if __name__ == "__main__":
    agent = DQLAgent(env)
    batch_size = 16
    episodes = 10

    for e in range(episodes):
        state = env.reset()
        if isinstance(state, tuple):
            state = state[0]
        state = np.reshape(state, [1, 4])
        time = 0

        while True:
            action = agent.act(state)
            next_state, reward, done, _, _ = env.step(action)
            next_state = np.reshape(next_state, [1, 4])
            agent.remember(state, action, reward, next_state, done)
            agent.replay(batch_size)
            agent.adaptiveEGreedy()
            state = next_state

            if done:
                print(f'Episode: {e}, Time: {time}')
                break
            time += 1

        if e % 5 == 0:
            agent.model.save('cartpole_dql.keras')

    print("Entrenamiento finalizado. Guardando modelo...")
    agent.model.save('cartpole_dql_final.keras')

# 🎥 GRABAR VIDEO DEL AGENTE
def record_video(env, agent, video_path="cartpole_video.mp4", frames=500):
    obs = env.reset()
    if isinstance(obs, tuple):
        obs = obs[0]
    obs = np.reshape(obs, [1, 4])

    frame_shape = (600, 400)
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, frame_shape)

    for _ in range(frames):
        frame = env.render()

        if frame is None:
            print("⚠️ Error: El frame renderizado es None.")
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = cv2.resize(frame, frame_shape)

        out.write(frame)

        action = np.argmax(agent.model.predict(obs, verbose=0))
        obs, _, done, _, _ = env.step(action)
        obs = np.reshape(obs, [1, 4])

        if done:
            break

    out.release()
    env.close()
    print("🎥 Video guardado correctamente en", video_path)

# 📌 LLAMAR A LA FUNCIÓN PARA GRABAR EL VIDEO
record_video(env, agent, "cartpole_video.mp4")

# 📌 FUNCIÓN PARA MOSTRAR EL VIDEO EN GOOGLE COLAB
def display_video(video_path):
    video_file = open(video_path, "rb").read()
    video_url = f"data:video/mp4;base64,{base64.b64encode(video_file).decode()}"
    return HTML(f'<video width="600" height="400" controls><source src="{video_url}" type="video/mp4"></video>')

# 📌 MOSTRAR EL VIDEO EN COLAB

!ffmpeg -i cartpole_video.mp4 -vcodec libx264 cartpole_video_fixed.mp4


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 6,555 B in 2s (4,079 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... D

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 0, Time: 19
Episode: 1, Time: 9
Episode: 2, Time: 17
Episode: 3, Time: 22
Episode: 4, Time: 33
Episode: 5, Time: 18
Episode: 6, Time: 19
Episode: 7, Time: 18
Episode: 8, Time: 55
Episode: 9, Time: 17
Entrenamiento finalizado. Guardando modelo...
🎥 Video guardado correctamente en cartpole_video.mp4
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopen